In [1]:
#Testing models

In [38]:
# -*- coding: UTF-8 -*-
# Local modules
import os
import sys
import argparse
# 3rd-Party Modules
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
import glob
import librosa
import copy

# PyTorch Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, DataLoader
import torch.optim as optim
from transformers import AutoModel, AutoFeatureExtractor, AutoProcessor
import importlib
# Self-Written Modules
sys.path.append(os.getcwd())
sys.path.append('../')
from benchmark import net
from benchmark import utils

In [11]:
# !pip install --upgrade transformers

In [8]:
import transformers
transformers.__version__

'4.47.1'

In [12]:
SSL = 'microsoft/wavlm-base'
ssl_model = AutoModel.from_pretrained(SSL)
processor = AutoFeatureExtractor.from_pretrained(SSL)
ssl_model.freeze_feature_encoder()
ssl_model.eval();

In [14]:
SSL = 'facebook/wav2vec2-large-robust'
ssl_model = AutoModel.from_pretrained(SSL)
processor = AutoFeatureExtractor.from_pretrained(SSL)
ssl_model.freeze_feature_encoder()
ssl_model.eval();

In [39]:
SSL = 'openai/whisper-large-v3'
ssl_model = AutoModel.from_pretrained(SSL)
processor = AutoProcessor.from_pretrained(SSL)
# ssl_model.freeze_feature_encoder()
ssl_model.eval();

In [36]:
# y, sr = librosa.load('../data/Audios/MSP-PODCAST_0835_0374_0004.wav', sr = None)
# with torch.no_grad():
#     mean = ssl_model(torch.tensor(y).unsqueeze(0)).last_hidden_state.squeeze(0)[0].mean()
# mean

In [45]:
y, sr = librosa.load('../data/Audios/MSP-PODCAST_0835_0374_0004.wav', sr = None)

inputs = processor(y, sampling_rate=sr, return_tensors="pt")

# Move inputs to GPU
input_features = inputs["input_features"]

with torch.no_grad():
    last = ssl_model.encoder(input_features).last_hidden_state
last.shape

torch.Size([1, 1500, 1280])

In [47]:
audio_length = len(y)
n_frames = int(np.ceil(audio_length / 320))

last.squeeze(0)[:n_frames, :].shape

torch.Size([298, 1280])

In [31]:
ssl_model(**inputs).last_hidden_state.shape

torch.Size([1, 297, 1024])

In [6]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
text_model = RobertaModel.from_pretrained("roberta-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [27]:
text_model.config.hidden_size

768

In [10]:
text = "Your input text here"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

In [11]:
with torch.no_grad():  # Disable gradient computation for inference
    outputs = text_model(**inputs)

In [15]:
outputs.last_hidden_state.shape

torch.Size([1, 6, 768])

In [16]:
out_wav = ssl_model(torch.rand((1,16000*4)))

In [18]:
out_wav.last_hidden_state.shape

torch.Size([1, 199, 768])

In [19]:
import torch.nn as nn

In [22]:
kernel_size = 3
cnn1 = nn.Conv1d(
            in_channels=768,  # Input channels = hidden size of RoBERTa
            out_channels=256,
            kernel_size=kernel_size,
            padding=kernel_size // 2  # Padding to maintain sequence length
        ).to(device)

cnn2 = nn.Conv1d(
            in_channels=768,  # Input channels = hidden size of RoBERTa
            out_channels=256,
            kernel_size=kernel_size,
            padding=kernel_size // 2  # Padding to maintain sequence length
        ).to(device)

In [24]:
text_out = cnn1(outputs.last_hidden_state.permute(0,2,1).to(device))
wav_out = cnn2(out_wav.last_hidden_state.permute(0,2,1).to(device))

print(text_out.shape, wav_out.shape)

torch.Size([1, 256, 6]) torch.Size([1, 256, 199])


In [26]:
maxpool = nn.AdaptiveMaxPool1d(1)
# m_text = nn.MaxPool1d(3, stride=2)
# m_audio = m = nn.MaxPool1d(7, stride=5)

text_out2 = maxpool(text_out)
wav_out2 = maxpool(wav_out)

print(text_out2.shape, wav_out2.shape)

torch.Size([1, 256, 1]) torch.Size([1, 256, 1])


In [6]:
# ssl_model.config.hidden_size

In [7]:
# torch.cuda.empty_cache()
# del transcriber

In [8]:
from transformers import pipeline
import numpy as np

# transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3", device=0, batch_size= 64)

from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

# Load model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3").to("cuda")
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Disable gradient computation
model.eval()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0): WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
    

In [9]:

import logging

In [10]:
df = pd.read_csv('../benchmark/processed_labels.csv')
df.head(2)

,FileName,Angry,Sad,Happy,Surprise,Fear,Disgust,Contempt,Neutral,Split_Set
0,MSP-PODCAST_0002_0033.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development
1,MSP-PODCAST_0002_0039.wav,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Development


In [11]:
!ls ../data/Labels/labels_consensus.csv

../data/Labels/labels_consensus.csv


In [12]:
df = pd.read_csv('../data/Labels/labels_consensus.csv')
df.head(2)

,FileName,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_0002_0033.wav,N,4.8,4.2,5.4,127,Female,Development
1,MSP-PODCAST_0002_0039.wav,N,4.0,4.2,4.2,127,Female,Development


In [13]:
def transcribe(audio_files, batch_size=8):
    results = []
    for i in range(0, len(audio_files), batch_size):
        batch = audio_files[i:i + batch_size]

        # Prepare inputs
        inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)

        # Move inputs to GPU
        input_features = inputs["input_features"].to("cuda")

        # Perform inference
        with torch.no_grad():
            output = model.generate(input_features)

        # Decode and store results
        transcriptions = processor.batch_decode(output, skip_special_tokens=True)
        results.extend(transcriptions)
    return results

In [14]:
# # Prepare inputs
# inputs = processor(batch, sampling_rate=16000, return_tensors="pt", padding=True)

# # Move inputs to GPU
# input_features = inputs["input_features"].to("cuda")

# # Perform inference
# with torch.no_grad():
#     output = model.generate(input_features)

# # Decode and store results
# transcriptions = processor.batch_decode(output, skip_special_tokens=True)

In [15]:
sample = f"../data/Audios/{df.FileName.values[1]}"

y, sr = librosa.load(sample)

# result = transcriber({"sampling_rate": sr, "raw": y})["text"]  

inputs = processor([y,y], sampling_rate=16000, return_tensors="pt")#,  padding=True, max_length=3000)

# inputs["input_features"].shape

# # Move inputs to GPU
input_features = inputs["input_features"].to("cuda")

# # # Perform inference
with torch.no_grad():
    output = model.generate(input_features)

# # # Decode and store results
transcriptions = processor.batch_decode(output, skip_special_tokens=True)
transcriptions

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[" 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes.",
 " 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes."]

In [16]:
def transcribe_it(y):
    # y, sr = librosa.load(w_path)

    # result = transcriber({"sampling_rate": sr, "raw": y})["text"]  
    
    inputs = processor(y, sampling_rate=16000, return_tensors="pt")#,  padding=True, max_length=3000)
    
    # inputs["input_features"].shape
    
    # # Move inputs to GPU
    input_features = inputs["input_features"].to("cuda")
    
    # # # Perform inference
    with torch.inference_mode():
        output = model.generate(input_features)

    del input_features 
    
    # # # Decode and store results
    transcriptions = processor.batch_decode(output, skip_special_tokens=True)
    return transcriptions

In [17]:
 y, sr = librosa.load(sample)

transcribe_it(y)

[" 1500 or something. It talks about the two different medical devices that they're authorizing them to use. And part one of them is for them to allow these chips to go into our bodies for medical purposes."]

In [18]:
wav_files = [f"../data/Audios/{f}" for f in df.FileName]
file_ids = df.FileName

In [19]:
import os

for w in wav_files:
    if(not os.path.isfile(w)):
        print('error')
        break

In [20]:
# !pip install datasets

In [ ]:
# Helper function to load audio files
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=16000)  # Ensure 16kHz sampling rate
    return audio

batch_size = 32
import tqdm
import gc
# Process in batches
results = []
c = 0
for i in tqdm.tqdm(range(0, len(wav_files), batch_size)):
    model.past_key_values = None
    batch_files = wav_files[i:i + batch_size]
    batch_ids = file_ids[i:i + batch_size]

    # Load audio files for the current batch
    audios = [load_audio(file) for file in batch_files]

    # Get transcriptions
    transcriptions = transcribe_it(audios)

    # torch.cuda.empty_cache()
    # torch.cuda.empty_cache()
    # gc.collect()
    # torch.cuda.synchronize()
    # gc.collect()

    if(c%5==0):
        torch.cuda.empty_cache()
        gc.collect()
        # torch.cuda.reset_peak_memory_stats()
        # Optional: Force CUDA device reset
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()

    if(c%10==0):
        del model
        model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3").to("cuda")
        
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB // GPU memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    # del audios
    # transcriber.model.to("cpu")
    # torch.cuda.empty_cache()
    # transcriber.model.to("cuda")
    # Merge transcriptions with IDs
    for trans, file_id in zip(transcriptions, batch_ids):
        results.append({"FileName": file_id, "transcription": trans})

    c += 1

  0%|                                                                               | 1/3632 [00:24<24:36:52, 24.40s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 12.98 GB


  0%|                                                                               | 2/3632 [00:46<22:58:37, 22.79s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.81 GB


  0%|                                                                               | 3/3632 [01:08<22:39:37, 22.48s/it]

...

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.81 GB


  2%|█▊                                                                            | 83/3632 [32:45<22:27:35, 22.78s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.81 GB


  2%|█▊                                                                            | 84/3632 [33:07<22:22:24, 22.70s/it]

GPU memory allocated: 12.62 GB // GPU memory reserved: 29.81 GB


In [ ]:
for i in tqdm.tqdm(range(0, len(wav_files), batch_size)):
    batch_files = wav_files[i:i + batch_size]
    batch_ids = file_ids[i:i + batch_size]

In [ ]:
# del transcriber

In [ ]:
# import pandas as pd

# # Convert results to DataFrame
# transcriptions_df = pd.DataFrame(results)
# transcriptions_df.head()

In [ ]:
# transcriptions_df.to_csv("whisper_transcripts.csv", index = False)

In [7]:
import pandas as pd
transcriptions_df = pd.read_csv("whisper_transcripts.csv")
transcriptions_df.head(),transcriptions_df.shape

(                    FileName  \
 0  MSP-PODCAST_0002_0033.wav   
 1  MSP-PODCAST_0002_0039.wav   
 2  MSP-PODCAST_0002_0051.wav   
 3  MSP-PODCAST_0002_0059.wav   
 4  MSP-PODCAST_0002_0061.wav   
 
                                        transcription  
 0               and I mean the numbers, right? So...  
 1   1500 or something. It talks about the two dif...  
 2   I did. So here it is. So it's page 1501 in th...  
 3   You know, it sounds like it could be like a b...  
 4   Yeah. So, but Molly, I mean, this is really o...  ,
 (116221, 2))

In [5]:
transcriptions_df['FileName'] = transcriptions_df['id'] 

In [8]:
# transcriptions_df[['FileName','transcription']].to_csv("whisper_transcripts.csv", index = False)

In [ ]:
!nvidia-smi

In [ ]:

torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
# Optional: Force CUDA device reset
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

In [ ]:
def force_gpu_cleanup():
    # Delete all CUDA tensors
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    del obj
        except Exception:
            pass
    
    # Clear CUDA cache
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    # Force Python garbage collection
    gc.collect()
    
    # Optional: Reset CUDA device
    torch.cuda.reset_peak_memory_stats()
    if torch.cuda.is_available():
        current_device = torch.cuda.current_device()
        torch.cuda.set_device(current_device)

In [ ]:
os.system('sync') # Linux/Mac only

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()
# Optional: Force CUDA device reset
# torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB // GPU memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")